<a href="https://colab.research.google.com/github/nurimammasri/Dicoding-Belajar-Pengembangan-Machine-Learning/blob/main/14_DeployWeb_NLP_TFJS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **binary classification dataset https://www.kaggle.com/marklvl/sentiment-labelled-sentences-data-set?**

Untuk mendownload data menggunakan API Kaggle, buat akun Kaggle. Pada kanan atas klik My Profile > Account pada titik tiga > API > Create API Account. Copykan Username & Key pada kode di bawah

In [1]:
# import os
# os.environ['KAGGLE_USERNAME'] = #UBAH DENGAN USERNAME ANDA
# os.environ['KAGGLE_KEY'] = #UBAH DENGAN PASSWORD ANDA

In [2]:
# upload kaggle.json
from google.colab import files
files.upload()
# make directory and change permission
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

Saving kaggle.json to kaggle.json
kaggle.json


In [3]:
!kaggle datasets download -d marklvl/sentiment-labelled-sentences-data-set

  0% 0.00/326k [00:00<?, ?B/s]
100% 326k/326k [00:00<00:00, 44.4MB/s]


In [4]:
!unzip -q sentiment-labelled-sentences-data-set.zip -d .

## **LSTM**

In [5]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv('/content/sentiment labelled sentences/yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
df.shape

(1000, 2)

In [6]:
df.head

<bound method NDFrame.head of                                               sentence  label
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]>

Text Preprocessing

In [7]:
# convert to lowercase
df['sentence'] = df['sentence'].str.lower()

In [8]:
# remove stopwords

from nltk.corpus import stopwords #comment jika Error dan gunakan 2 sintaks dibawah
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:

stop = set(stopwords.words('english'))
df['sentence'] = df['sentence'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop)]))
df.head()

,sentence,label
0,wow... loved place.,1
1,crust good.,0
2,tasty texture nasty.,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great prices.,1


Tokenize

In [10]:
vocab_size = 2000
oov_tok = "<OOV>"
filt = '!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ' #remove symbols

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok, filters = filt)
tokenizer.fit_on_texts(df['sentence'].values)

word2index = tokenizer.word_index
print(len(word2index))

1998


In [11]:
import json

with open('word2index.json', 'w') as fp:
    json.dump(word2index, fp)

In [12]:
max_length =  max(len(values.split()) for i, values in enumerate(df['sentence']))
max_length

18

In [13]:
trunc_type='post'

all_seq = tokenizer.texts_to_sequences(df['sentence'].values)
all_padded = pad_sequences(all_seq, maxlen = max_length, padding = trunc_type)
all_padded.shape

(1000, 18)

In [14]:
# split train and test sets
from sklearn.model_selection import train_test_split

X = all_padded
#y = pd.get_dummies(df['label'].values)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

#kalimat = df['sentence'].values
#y = df['label'].values

#kalimat_latih, kalimat_test, y_latih, y_test = train_test_split(kalimat, y, 
#                                                                test_size=0.2, random_state=1000)

(800, 18) (800,)
(200, 18) (200,)


In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim= vocab_size, output_dim=16, input_length= max_length),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 18, 16)            32000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                20736     
_________________________________________________________________
dense (Dense)                (None, 24)                1560      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 54,321
Trainable params: 54,321
Non-trainable params: 0
_________________________________________________________________


In [16]:
num_epochs = 30
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_test, y_test))

Epoch 1/30
25/25 [==============================] - 3s 30ms/step - loss: 0.6935 - accuracy: 0.4950 - val_loss: 0.6932 - val_accuracy: 0.4800
Epoch 2/30
25/25 [==============================] - 0s 11ms/step - loss: 0.6897 - accuracy: 0.5412 - val_loss: 0.6830 - val_accuracy: 0.5800
Epoch 3/30
25/25 [==============================] - 0s 12ms/step - loss: 0.4323 - accuracy: 0.8363 - val_loss: 0.6730 - val_accuracy: 0.7300
Epoch 4/30
25/25 [==============================] - 0s 12ms/step - loss: 0.1664 - accuracy: 0.9488 - val_loss: 0.6557 - val_accuracy: 0.7300
Epoch 5/30
25/25 [==============================] - 0s 11ms/step - loss: 0.0780 - accuracy: 0.9775 - val_loss: 0.6820 - val_accuracy: 0.7500
Epoch 6/30
25/25 [==============================] - 0s 11ms/step - loss: 0.0395 - accuracy: 0.9875 - val_loss: 0.9372 - val_accuracy: 0.7450
Epoch 7/30
25/25 [==============================] - 0s 11ms/step - loss: 0.0424 - accuracy: 0.9875 - val_loss: 0.9219 - val_accuracy: 0.7450
Epoch 8/30
25

In [17]:
#def toSequence(sentence):
#  pad = []
#  for stc in sentence.split():
#    if stc.lower() in word2index.keys(): 
#      pad.append(word2index[stc.lower()])
#    else: 
#      continue
#  return pad

#pad = toSequence('affordable price and nice dessert')
#pad = [269, 353, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0 ,0,0,0,0]
#len(pad)
#model.predict([pad])

Save Model


In [18]:
!pip install tensorflowjs

     |████████████████████████████████| 64 kB 2.4 MB/s 


In [19]:
saved_model_path = '/content/mymodel/'
tf.saved_model.save(model, saved_model_path)

INFO:tensorflow:Assets written to: /content/mymodel/assets


INFO:tensorflow:Assets written to: /content/mymodel/assets


In [20]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  /content/mymodel/ \
  /content/modeltfjs

2021-09-17 06:33:21.628756: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-17 06:33:21.628811: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c821ba6fa100): /proc/driver/nvidia/version does not exist
2021-09-17 06:33:23.413188: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-09-17 06:33:23.413411: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2021-09-17 06:33:23.423609: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 251 nodes (239), 348 edges (336), time = 5.744ms.
  function_optimizer: function_optimizer did nothing. time = 0.085ms.

2021-09-17 06:33:23.632442: I tensorflow/core/grappler/optimizers/meta_optim